In [56]:
# part of power analysis function #
# assuming I can get an array of SEs #

#import mne
import numpy as np
import pandas as pd
import os
import glob
from matplotlib import pyplot as plt
import sys
import warnings
import seaborn as sns
from itertools import product as prod

# temporary 
from scipy import stats
from statsmodels.stats.power import ttest_power, tt_ind_solve_power



In [156]:
# create arrays of SEs, using 4 electrodes

arr_simp = np.array([.01,1,2,10])

# fake subjects and effect sizes
Subs = [20,25,30,35]
effect = [3, 5]


In [157]:
# t tests with simple array

t_arr = np.zeros((len(arr_simp), len(Subs), len(effect)))
p_arr = np.zeros((len(arr_simp), len(Subs), len(effect)))

for ele in range(len(arr_simp)):  # loop over electrodes
    BESE = arr_simp[ele]
    
    for s in range(len(Subs)):    # loop over sample sizes
        n = Subs[s]
        df = 2*n - 2              # get degrees of freedom (based on sample size)
        
        for ef in range(len(effect)):          # loop over effect sizes
            t = effect[ef] / (BESE/np.sqrt(n)) # get t value 
            t_arr[ele][s][ef] = t              # save t value
            
            p = 1 - stats.t.cdf(t,df=df)       # get p value
            p_arr[ele][s][ef] = p              # save p value


In [158]:
# t values
print(t_arr)

[[[1.34164079e+03 2.23606798e+03]
  [1.50000000e+03 2.50000000e+03]
  [1.64316767e+03 2.73861279e+03]
  [1.77482393e+03 2.95803989e+03]]

 [[1.34164079e+01 2.23606798e+01]
  [1.50000000e+01 2.50000000e+01]
  [1.64316767e+01 2.73861279e+01]
  [1.77482393e+01 2.95803989e+01]]

 [[6.70820393e+00 1.11803399e+01]
  [7.50000000e+00 1.25000000e+01]
  [8.21583836e+00 1.36930639e+01]
  [8.87411967e+00 1.47901995e+01]]

 [[1.34164079e+00 2.23606798e+00]
  [1.50000000e+00 2.50000000e+00]
  [1.64316767e+00 2.73861279e+00]
  [1.77482393e+00 2.95803989e+00]]]


In [159]:
# p values
print ( p_arr)

[[[0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]]

 [[3.33066907e-16 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]]

 [[3.06019163e-08 7.02771175e-14]
  [6.32174757e-10 1.11022302e-16]
  [1.33266731e-11 0.00000000e+00]
  [2.84883228e-13 0.00000000e+00]]

 [[9.38377405e-02 1.56487342e-02]
  [7.00818569e-02 7.94484547e-03]
  [5.28795835e-02 4.09079077e-03]
  [4.02012659e-02 2.12778133e-03]]]


In [154]:
# fake arrays

Subs = [1,10,20,50]
effects = [1, 2, 3, 5]
arr_simp = np.array([1,2,3,5])


In [146]:
pwrs = np.zeros((len(arr_simp), len(Subs), len(effects)))

for ele in range(len(arr_simp)):  # loop over electrodes
    BESE = arr_simp[ele]
    
    for s in range(len(Subs)):    # loop over sample sizes
        n = Subs[s]
        SD = BESE/(np.sqrt(n))    # get SD from BESE (yeah maybe wrong)
        #null_dist = np.random.normal(loc = 0, scale = SD, size = 1000)     # make null distribution
        crit = 1.96*SD                                                      # get critical value
        
        for effect in range(len(effects)):                                  # loop over effect sizes
            ef = effects[effect]
            alt_dist = np.random.normal(loc = ef, scale = SD, size = 10000) # make alternative distribution 
            pwr = len(alt_dist[alt_dist>crit])/len(alt_dist)
            
            pwrs[ele][s][effect] = pwr              # save power 


In [147]:
pwrs

array([[[0.1678, 0.5066, 0.8512, 0.9991],
        [0.8835, 1.    , 1.    , 1.    ],
        [0.9945, 1.    , 1.    , 1.    ],
        [1.    , 1.    , 1.    , 1.    ]],

       [[0.0709, 0.1704, 0.3196, 0.7103],
        [0.3469, 0.8848, 0.9965, 1.    ],
        [0.6056, 0.994 , 1.    , 1.    ],
        [0.9451, 1.    , 1.    , 1.    ]],

       [[0.0521, 0.0967, 0.1738, 0.388 ],
        [0.1737, 0.5571, 0.8815, 0.9998],
        [0.3133, 0.8452, 0.9944, 1.    ],
        [0.6678, 0.9972, 1.    , 1.    ]],

       [[0.0391, 0.0563, 0.088 , 0.1734],
        [0.0893, 0.2424, 0.4809, 0.8889],
        [0.1408, 0.4351, 0.7619, 0.9943],
        [0.2837, 0.809 , 0.9887, 1.    ]]])